### 폴더 탐색기에서 폴더 선택하여 폴더 안의 파일 리스트 가져오기

In [ ]:
from tkinter import filedialog
import os
import re

# 폴더/파일 탐색기에서 폴더 경로 가져오기
TPath = filedialog.askdirectory()
# 파워포인트 파일 리스트 가져오기
pptfiles = [f for f in os.listdir(TPath) if re.match(r'.*[.]ppt', f)]
# 엑셀 파일 리스트 가져오기
xlfiles = [f for f in os.listdir(TPath) if re.match(r'.*[.]xls', f)]
# 워드 파일 리스트 가져오기
wordfiles = [f for f in os.listdir(TPath) if re.match(r'.*[.]doc', f)]
# Image 파일 리스트 불러오기, 확장자명에 jpg, png, gif가 포함된 파일 전체
imgfiles = [f for f in os.listdir(TPath) if re.match('.*([.]jpg|[.]png|[.]gif)', f)]


### pywin32에서 애플리케이션 실행하기

In [ ]:
import win32com.client

# 파워포인트 실행하기
powerpoint = win32com.client.Dispatch("Powerpoint.Application")
prs = powerpoint.Presentations.Open(‘파일 경로’)
prs.Close()
powerpoint.Quit()

# 엑셀 실행하기
excel = win32com.client.Dispatch("Excel.Application")
wb = excel.Workbooks.Open(“파일 경로”)
wb.Close()
excel.Quit()

# 워드 실행하기
word = win32com.client.Dispatch("Word.Application")
doc = word.Documents.Open(“파일 경로”)
doc.Close()
word.Quit()

### os 라이브러리로 파일 이름 다루기

In [ ]:
import os
#폴더 경로 \\ 파일 이름 으로 구조 만들기
os.path.join("폴더경로","파일 이름") 
# 파일 이름을 이름과 확장자명 분리하여 pre와 ext에 저장
pre,ext = os.path.splitext("파일 이름") 

### python-pptx 개체 번호 따기

In [ ]:
from pptx import Presentation

prs = Presentation('PPT 파일 이름')
for idx, slide_layout in enumerate(prs.slide_layouts):
    slide = prs.slides.add_slide(slide_layout)
    for shape in slide.placeholders:
        print(idx, shape.placeholder_format.idx, shape.name, shape.top, shape.left)

### python-pptx 지정된 Layout으로 Slide 삽입하고 placeholder에 개체 삽입하기

In [ ]:
from pptx import Presentation

prs = Presentation('PPT 파일 이름')
slide_layout = prs.slide_layouts["Layout고유번호(int)"]

sld = prs.slides.add_slide(slide_layout)
sld.placeholders["고유번호(idx)"].insert_picture("이미지파일 이름(경로포함)")
sld.placeholders["고유번호(idx)"].text = "입력할 Text"

### python-pptx 텍스트 프레임 내부 run까지 따져 편집하기	

In [ ]:
from pptx import Presentation
from pptx.enum.shapes import MSO_SHAPE_TYPE

def para_function (shp):
    for paragraph in shp.text_frame.paragraphs:
        # 문단 단위로 다룰 내용이 있으면 관련 내용 수정
        for run in paragraph.runs:
            # 단어 단위로 다룰 내용이 있으면 관련 내용 수정

# 파워포인트에서 파일 읽어오기
prs = Presentation(“파일 경로”)
for slide in prs.slides:
      for shape in slide.shapes:
            # shape 단에서 다룰 내용이 있으면, shape 관련 내용 수정

            # shape에 Text Frame이 포함되어 있는 경우
            if shape.has_text_frame:
                  # text frame이 포함된 shape에서만 다룰 내용이 있으면 관련 내용 수정
                  para_function (shape)

            # shape가 표인 경우
            if shape.has_table:
                  # table이 포함된 shape에서만 다룰 내용이 있으면 관련 내용 수정

                  for row in shape.table.rows:
                        # 표의 행 단위로 다룰 내용이 있으면 관련 내용 수정

                        for cell in row.cells:
                              # 표 안의 각 칸 단위로 다룰 내용이 있으면 관련 내용 수정
                              para_function (cell)

            # shape가 Group으로 묶인 경우
            if shape.shape_type == MSO_SHAPE_TYPE.GROUP:

                  for shp in shape.shapes:
                        if shp.has_text_frame:
                              # text frame이 포함된 shape에서만 다룰 내용 수정
                              para_function (shp)

### pywin32 파워포인트 텍스트 프레임 내부까지 따져 편집하기

In [ ]:
import win32com.client

# 파워포인트 애플리케이션 개체 만들기
powerpoint = win32com.client.Dispatch("Powerpoint.Application")

# 파워포인트에서 파일 읽어오기
ppt = powerpoint.Presentations.Open(“파일 경로”)

# 모든 slide에 대해 반복
for slide in ppt.Slides:
    # 모든 shape에 대해 반복
    for shape in slide.shapes:
        # TextFrame이 있는 경우
        if shape.HasTextFrame == -1:
            # Text Frame 내부 다룰 내용

        # 표가 있는 경우
        elif shape.HasTable == -1:
            # 모든 행에 대하여 반복
            for row in shape.Table.Rows:
                # 행을 구성하는 모든 셀에 대하여 반복
                for cell in row.cells:
                    # Text Frame 내부 다룰 내용

        # 그 외의 경우
        else:
            # try 아래 구문은 시도하다가 에러가 발생하거나 실패할 경우 except로 이동
            try:
                # Group을 구성하는 모든 구성 Shape에 대해 반복. 
                # 개체가 Group이 아닐때 에러
                for GI in shape.GroupItems:
                    if GI.HasTextFrame == -1:
                        # Text Frame 내부 다룰 내용

            # 에러 발생 시 다음 단계로 이동
            except:
                pass

### python-docx 텍스트 내부 run까지 따져 편집하기

In [ ]:
from docx import Document

# 워드 파일 읽어오기
doc = Document(“파일 경로”)

# 각 문단에서 반복문 실행
for paragraph in doc.paragraphs:
    # 문단의 각 run(띄어쓰기) 단위로 반복문 실행
    for run in paragraph.runs:
        # run 단위 다룰 내용

# 각 표에서 반복문 실행
for table in doc.tables:
    # 표의 각 행단위로 반복문 실행
    for row in table.rows:
        # 행 내부의 Cell 단위로 반복문 실행
        for cell in row.cells:
            # cell 단위 다룰 내용